# Conversational Memory

This is how a chatbop can respond to multiple queries in a chat-like manner. Previous interactions allow context to be captured from previous interactions.

In [5]:
# load keys
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain import OpenAI
from langchain.chains import ConversationChain

# initialize the LLM
llm = OpenAI(
temperature=0,
model_name="text-davinci-003")

# initialize the conversation chain
conversation = ConversationChain(llm=llm)

print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## Forms of Conversations Memory

### Conversational Buffer Memory


In [8]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
llm=llm,
memory=ConversationBufferMemory()
)

conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today, isn't it? How can I help you?"}

In [13]:
from langchain.callbacks import get_openai_callback

# keep track of tokens spent
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
        return result

In [14]:
count_tokens(
conversation_buf,
"My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [15]:
count_tokens(
conversation_buf,
"I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 264 tokens


' There are many possibilities for integrating Large Language Models with external knowledge. For example, you could use external knowledge to provide additional context to the model, or to provide additional training data. You could also use external knowledge to help the model better understand the context of a given text, or to help it generate more accurate results.'

In [16]:
count_tokens(
conversation_buf,
"Which data source types could be used to give context to the model?"
)

Spent a total of 344 tokens


'  Data sources that could be used to give context to the model include text corpora, images, audio, and video. Text corpora can provide additional context to the model by providing additional training data. Images, audio, and video can provide additional context by providing additional visual or auditory information.'

In [17]:
count_tokens(
conversation_buf,
"What is my aim again?"
)

Spent a total of 372 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge.'

In [18]:
# how is the memory stored?
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI:  Good morning! It's a beautiful day today, isn't it? How can I help you?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?
Human: I just want to analyze the different possibilities. What can you think of?
AI:  There are many possibilities for integrating Large Language Models with external knowledge. For example, you could use external knowledge to provide additional context to the model, or to provide additional training data. You could also use external knowledge to help the model bette

### Conversation Summary Memory

This form of memory summarizes the conversation history before it is passed to the {history} param. This reduces the number of tokens used.

In [20]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
llm=llm,
memory=ConversationSummaryMemory(llm=llm)
)

print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [21]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
conversation_sum,
"Good morning AI!"
)

Spent a total of 290 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [22]:
count_tokens(
conversation_sum,
"My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 440 tokens


" That sounds like an interesting project! I'm familiar with Large Language Models, but I'm not sure how they could be integrated with external knowledge. Could you tell me more about what you have in mind?"

In [23]:
count_tokens(
conversation_sum,
"I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 683 tokens


' I can think of a few possibilities. One option is to use a large language model to generate a set of candidate answers to a given query, and then use external knowledge to filter out the correct answer. Another option is to use the large language model to generate a set of candidate answers, and then use external knowledge to score each answer and select the one with the highest score. Finally, you could use the large language model to generate a set of candidate answers, and then use external knowledge to refine the answers and provide additional context.'

In [24]:
count_tokens(
conversation_sum,
"Which data source types could be used to give context to the model?"
)

Spent a total of 802 tokens


' There are many different types of data sources that could be used to give context to the model. These could include structured data sources such as databases, unstructured data sources such as text documents, or even external APIs that provide access to external knowledge. Additionally, the model could be trained on a combination of these data sources to provide a more comprehensive understanding of the context.'

In [25]:
count_tokens(
conversation_sum,
"What is my aim again?"
)

Spent a total of 856 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge.'

In [26]:
# what are we storing?
print(conversation_sum.memory.buffer)


The human greeted the AI with a good morning, to which the AI responded with a good morning and asked how it could help. The human expressed interest in exploring the potential of integrating Large Language Models with external knowledge, to which the AI responded positively and asked for more information. The AI then suggested a few possibilities, such as using the large language model to generate a set of candidate answers and then using external knowledge to filter out the correct answer, score each answer and select the one with the highest score, or refine the answers and provide additional context. The human then asked which data source types could be used to give context to the model, to which the AI responded that there are many different types of data sources that could be used, such as structured data sources, unstructured data sources, and external APIs. Additionally, the model could be trained on a combination of these data sources to provide a more comprehensive understan

We used more tokens in this example, but for long conversations, memorization of conversation history will use much more than a summarization approach.

### Conversation Buffer Window Memory

Same appraoch as Conversation Buffer but with a window so that older interactions are forgotten

In [30]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation_bufw = ConversationChain(
llm=llm,
memory=ConversationBufferWindowMemory(k=1)
)

# k=1 means the window will only include the one latest interaction

In [31]:
count_tokens(
conversation_bufw,
"Good morning AI!"
)

Spent a total of 85 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [32]:
count_tokens(
conversation_bufw,
"My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [33]:
count_tokens(
conversation_bufw,
"I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 233 tokens


' There are many possibilities for integrating Large Language Models with external knowledge. For example, you could use external knowledge to provide additional context to the model, or to provide additional training data. You could also use external knowledge to help the model better understand the context of a given text, or to help it generate more accurate results.'

In [34]:
count_tokens(
conversation_bufw,
"Which data source types could be used to give context to the model?"
)

Spent a total of 245 tokens


' Data sources that could be used to give context to the model include text corpora, structured databases, and ontologies. Text corpora provide a large amount of text data that can be used to train the model and provide additional context. Structured databases provide structured data that can be used to provide additional context to the model. Ontologies provide a structured representation of knowledge that can be used to provide additional context to the model.'

In [35]:
count_tokens(
conversation_bufw,
"What is my aim again?"
)

Spent a total of 186 tokens


' Your aim is to use data sources to give context to the model.'

In [37]:
# what are we saving?
bufw_history = conversation_bufw.memory.load_memory_variables(
inputs=[]
)['history']

print(bufw_history)

Human: What is my aim again?
AI:  Your aim is to use data sources to give context to the model.


In [38]:
# k can be adjusted to balance token use with memorized interactions

### Conversation Summary Buffer Memory

- a mix of Conversation Summary Memory and Conversation Buffer Window Memory. It summarizes the earier interactions and maintains the max_token_limit

In [42]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

conversation_sum_bufw = ConversationChain(
llm=llm, memory=ConversationSummaryBufferMemory(
llm=llm,
max_token_limit=650
))

# the max token limit will manage what is stored in memory in summary vs memorized